<a href="https://colab.research.google.com/github/kumaranu7/Tweet-Sentiments-analysis-using-BERT/blob/master/bERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import  numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub


In [0]:
tf.compat.v1.enable_eager_execution() # to prevent the error " 'Tensor' object has no attribute 'numpy' ".

In [5]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.6MB/s 


In [0]:
from bert import tokenization

In [0]:
train = pd.read_csv('/content/drive/My Drive/bert/train.csv')
test = pd.read_csv('/content/drive/My Drive/bert/test.csv')

In [9]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 8.68 s, sys: 1.69 s, total: 10.4 s
Wall time: 22.3 s


In [11]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [0]:
def pre_process(doc, tokenizer, max_len = 512):
  all_tokens = []
  all_mask = []
  all_segments = []

  for data in doc:
    review = re.sub('[^a-z A-Z]', ' ', data)
    url = re.compile('https?://\S+|www\.\S+')
    review = url.sub(r'', review)
    html = re.compile('<.*?>')
    review = html.sub(r'', review)

    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    review = emoji_pattern.sub(r'', review)
    text = tokenizer.tokenize(review)

    text = text[:max_len-2]
    ip_seq = ['[CLS]'] + text + ['[SEP]']
    pad_len = max_len - len(ip_seq)

    tokens = tokenizer.convert_tokens_to_ids(ip_seq)
    tokens += [0]*pad_len
    pad_mask = [1]*len(ip_seq) + [0]*pad_len

    segmend_id = [0]*max_len

    all_tokens.append(tokens)
    all_mask.append(pad_mask)
    all_segments.append(segmend_id)

  return (np.array(all_tokens), np.array(all_mask), np.array(all_segments))

In [0]:
max_len = 512

In [14]:
input_word_id = Input(shape=(max_len ,),dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(max_len ,), dtype=tf.int32, name="input_mask")
segment_id = Input(shape=(max_len ,), dtype=tf.int32, name = "segment_id")

_, seq_op = bert_layer([input_word_id, input_mask, segment_id])
clf_output = seq_op[:, 0, :]
model = Model(inputs = [input_word_id, input_mask, segment_id], outputs = clf_output)
model.compile(Adam(lr=2e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
segment_id (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [15]:
print('shape of _ layer of BERT' + str(_.shape))
print('shape of last layer of BERT' + str(seq_op))

shape of _ layer of BERT(?, 768)
shape of last layer of BERTTensor("keras_layer/Identity_1:0", shape=(?, ?, 768), dtype=float32)


In [0]:
def bert_model(bert_layer, max_len = 512):
  input_word_id = Input(shape=(max_len,),dtype=tf.int32, name="input_word_ids")
  input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
  segment_id = Input(shape=(max_len,), dtype=tf.int32, name = "segment_id")
  
  _, sequence_output = bert_layer([input_word_id, input_mask, segment_id])
  clf_output = sequence_output[:, 0, :]
  dense_1 = Dense(256, 'relu')(clf_output)
  dense_1 = Dropout(0.4)(dense_1)
  dense_2 = Dense(128, activation='relu')(dense_1)
  dense_2 = Dropout(0.4)(dense_2)
  out = Dense(1, activation='sigmoid')(dense_2)

  model = Model(inputs = [input_word_id, input_mask, segment_id], outputs = out)
  model.compile(Adam(lr = 2e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [0]:
train_ip = pre_process(train.text.values, tokenizer, max_len = 260)
test_ip = pre_process(test.text.values, tokenizer, max_len = 260)
train_label = train.target.values


In [18]:
model = bert_model(bert_layer, max_len=260)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 260)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 260)]        0                                            
__________________________________________________________________________________________________
segment_id (InputLayer)         [(None, 260)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 109482241   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [19]:
checkpoint = ModelCheckpoint('model.h5',  monitor = 'val_loss', save_best_only=True)
train_history = model.fit(train_ip, train_label, validation_split=0.2, epochs=10, callbacks= [checkpoint], batch_size=32)

Train on 6090 samples, validate on 1523 samples
Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


6090/6090 [==============================] - 221s 36ms/sample - loss: 0.4719 - acc: 0.7947 - val_loss: 0.4021 - val_acc: 0.8267
Epoch 2/10
6090/6090 [==============================] - 202s 33ms/sample - loss: 0.3451 - acc: 0.8693 - val_loss: 0.3934 - val_acc: 0.8332
Epoch 3/10
6090/6090 [==============================] - 198s 32ms/sample - loss: 0.2298 - acc: 0.9179 - val_loss: 0.4535 - val_acc: 0.8352
Epoch 4/10
6090/6090 [==============================] - 198s 32ms/sample - loss: 0.1398 - acc: 0.9498 - val_loss: 0.5996 - val_acc: 0.8207
Epoch 5/10
6090/6090 [==============================] - 198s 32ms/sample - loss: 0.0858 - acc: 0.9688 - val_loss: 0.6480 - val_acc: 0.8234
Epoch 6/10
6090/6090 [==============================] - 198s 33ms/sample - loss: 0.0572 - acc: 0.9760 - val_loss: 0.9249 - val_acc: 0.8122
Epoch 7/10
6090/6090 [==============================] - 198s 33ms/sample - loss: 0.0419 - acc: 0.9814 - val_loss: 1.1023 - val_acc: 0.8221
Epoch 8/10
6090/6090 [================

In [0]:
model.load_weights('model.h5')


In [0]:
predictions = model.predict(test_ip)